# Sarawak Energy Use Case

# Data Loading and Exploration

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
data_source_dir = r"K:\Projects\sarawak_energy\data"

In [3]:
def GetFile(file, directory=data_source_dir):
    return os.path.join(directory, file)

In [4]:
xls = pd.ExcelFile(GetFile("sarawak_energy.xlsx"))

In [5]:
xls.sheet_names

['Consumption_Point_Data_Package',
 'parameters',
 'Occurrencies_Data_Package',
 'Billing_Data_Package']

In [6]:
df_dict = {}
for sheet_name in xls.sheet_names:
    df_dict[sheet_name] = xls.parse(sheet_name)

In [7]:
df_dict.keys()

dict_keys(['Consumption_Point_Data_Package', 'parameters', 'Occurrencies_Data_Package', 'Billing_Data_Package'])

## File 1: Consumption Point Data Package

My thoughts on what this df is about:

* ID level aggregation of multiple mapping columns
* Each id has their own region, local, and municipality, granularity in order
* Assuming **was_fraud** is the target variable
* Fraud rate is about **13.2%**

In [11]:
df_cp = df_dict['Consumption_Point_Data_Package']
df_cp.head()

id   consumer_name  coordinate_lon  coordinate_lat  id_region  \
0  301075238101  TANG CHONG NGA             NaN             NaN     330101   
1  301045595109   WONG AK SELAN             NaN             NaN     330101   
2  214120301100  CHIN TECK NYUK             NaN             NaN     321401   
3  205150724106    CHAI MUI YAW             NaN             NaN     320501   
4  205146257100    BONG KIM KON             NaN             NaN     320501   

  id_locality id_municipality  id_uc_meter_type  id_risk_area  \
0    301P0085        301U0216               NaN           NaN   
1    301P0057        301U0137               NaN           NaN   
2    214P0006        214U0032               NaN           NaN   
3    205P0083        205U0409               NaN           NaN   
4    205P0081        205U0208               NaN           NaN   

  id_uc_aggregated_class      ...           MoveIn              MoveOut  \
0                    COM      ...       2002-04-10  9999-12-31 00:00:00   
1                    RES      ...       2002-04-10  9999-12-31 00:00:00   
2                    RES      ...       2002-04-03  2008-06-18 00:00:00   
3                    RES      ...       2002-03-20  9999-12-31 00:00:00   
4                    RES      ...       2002-03-19  9999-12-31 00:00:00   

  id_uc_phase id_uc_meter_brand id_uc_voltage_level   meter_id  \
0         NaN     SECURE METERS                 NaN   10P58085   
1         NaN              AMPY                 NaN  20S189641   
2         NaN               NaN                 NaN  10S090601   
3         NaN     SECURE METERS                 NaN   10P37494   
4         NaN     SECURE METERS                 NaN   10P42329   

  connection_date  was_fraud address billing_factor  
0      2002-04-10        NaN     NaN              1  
1      2002-04-10        NaN     NaN              1  
2      2002-04-03        NaN     NaN              1  
3      2002-03-20        NaN     NaN              1  
4      2002-03-19        NaN     NaN              1  

[5 rows x 23 columns]

In [12]:
df_cp.shape

(379, 23)

In [15]:
len(df_cp["id"].unique())

350

In [16]:
df_cp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 379 entries, 0 to 378
Data columns (total 23 columns):
id                          379 non-null int64
consumer_name               379 non-null object
coordinate_lon              0 non-null float64
coordinate_lat              0 non-null float64
id_region                   379 non-null int64
id_locality                 379 non-null object
id_municipality             379 non-null object
id_uc_meter_type            0 non-null float64
id_risk_area                0 non-null float64
id_uc_aggregated_class      379 non-null object
id_uc_class                 379 non-null object
id_uc_group                 0 non-null float64
id_uc_contract_situation    379 non-null object
MoveIn                      379 non-null datetime64[ns]
MoveOut                     379 non-null object
id_uc_phase                 0 non-null float64
id_uc_meter_brand           371 non-null object
id_uc_voltage_level         0 non-null float64
meter_id                    379 

### Data Cleansing Needed

In [49]:
non_null_cols = df_cp.isna().all()[df_cp.isna().all()==False].index.tolist()

df_cp = df_cp[non_null_cols]

df_cp.drop(["MoveIn", "MoveOut"], inplace=True, axis = 1)

### Fraud Rate

In [45]:
df_cp['was_fraud'].value_counts()['S'] / df_cp.shape[0] * 100

13.192612137203167

## File 2: Parameters

* mapping parameters for tariffs that correspond to the specific class
* class - commercial, residential, industrial, public

In [38]:
df_dict['parameters']

Tariff Class
0     C1   COM
1     C2   COM
2     C3   COM
3    DOM   RES
4     I1   IND
5     I2   IND
6     I3   IND
7     PL   PUB

## File 3: Occurrencies Data Package

* Activities done on the meters of the customer
* Wide range of activities, maybe text mine and one hot encode
* Possible features: number of inspections
* contains more ids than first file, 379 vs 1718
* date_executed contains a lot of 00/00/00. Perhaps it is the date where the activity really happened as was carried out
* Might need to feed it into **OpenRefine**

In [46]:
df_op = df_dict['Occurrencies_Data_Package']

In [56]:
df_op.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2038 entries, 0 to 2037
Data columns (total 8 columns):
id               2038 non-null int64
id_uc            2038 non-null int64
id_so_type       2038 non-null object
Desc             2038 non-null object
SystStatus       2038 non-null object
id_so_status     2038 non-null object
date_created     2038 non-null datetime64[ns]
date_executed    2038 non-null object
dtypes: datetime64[ns](1), int64(2), object(5)
memory usage: 127.5+ KB


In [53]:
non_null_cols = df_op.isna().all()[df_op.isna().all()==False].index.tolist()

df_op = df_op[non_null_cols]

In [52]:
non_null_cols

['id',
 'id_uc',
 'id_so_type',
 'Desc',
 'SystStatus',
 'id_so_status',
 'date_created',
 'date_executed']

In [55]:
len(df_op["id"].unique())

1718

In [58]:
df_op.head()

id         id_uc id_so_type  \
0  9000012040  100001141965         Z9   
1  9000011649  100001771213         Z9   
2  9000011518  100004801563         Z9   
3  9000011062  100004161914         Z9   
4  9000010687  100003721183         Z9   

                                       Desc SystStatus id_so_status  \
0            CONFIRM READING + 100001141965  NOCO NOPT    Completed   
1  CONFIRM READING + CA 100001771213 CT MTR  NOCO NOPT    Completed   
2                    RECONFIRMATION READING  NOCO NOPT    Completed   
3                METER READING CONFIRMATION  NOCO NOPT    Completed   
4              Meter Reading Reconfirmation  NOCO NOPT    Completed   

  date_created date_executed  
0   2018-08-08    00/00/0000  
1   2018-04-19    00/00/0000  
2   2018-03-14    00/00/0000  
3   2017-11-27    00/00/0000  
4   2017-09-14    00/00/0000

## File 4: Billing Data Package

* Monthly time series of consumption and billing
* Has 350 unique ids. Hence probably same bunch of customers from the first file
* 379 rows are null, might want to check if this has anything to do with fraud

In [61]:
df_bp = df_dict["Billing_Data_Package"]

In [62]:
df_bp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20135 entries, 0 to 20134
Data columns (total 10 columns):
id_uc                   20135 non-null int64
date_reference          20135 non-null datetime64[ns]
Read                    20135 non-null int64
Billed                  20135 non-null int64
consumption_invoiced    19756 non-null float64
consumption_measured    19756 non-null float64
date_measured           20135 non-null datetime64[ns]
value_measured          20135 non-null int64
ReadType                20135 non-null int64
id_billing_event        0 non-null float64
dtypes: datetime64[ns](2), float64(3), int64(5)
memory usage: 1.5 MB


In [66]:
len(df_bp["id_uc"].unique())

350

In [70]:
non_null_cols = df_bp.isna().all()[df_bp.isna().all()==False].index.tolist()

df_bp = df_bp[non_null_cols]

In [69]:
non_null_cols

['id_uc',
 'date_reference',
 'Read',
 'Billed',
 'consumption_invoiced',
 'consumption_measured',
 'date_measured',
 'value_measured',
 'ReadType']

In [72]:
df_bp.head()

id_uc date_reference  Read  Billed  consumption_invoiced  \
0  301075238101     2015-10-22   118     118                 118.0   
1  301075238101     2015-10-21     0       0                   NaN   
2  301075238101     2017-02-14   118     118                   0.0   
3  301045595109     2002-07-31  7722    7722                 391.0   
4  301045595109     2002-05-31  7182    7182                 401.0   

   consumption_measured date_measured  value_measured  ReadType  
0                 118.0    2015-10-22             118         1  
1                   NaN    2015-10-21               0         1  
2                   0.0    2017-02-14             118         1  
3                 391.0    2002-08-14            7722         1  
4                 401.0    2002-06-12            7182         1

In [77]:
df_bp[df_bp["consumption_invoiced"].isnull()].shape

(379, 9)